In [ ]:
# ===== 1. IMPORT THƯ VIỆN =====
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def train(ticker):
    try:
        df = pd.read_csv(f"E:/Project/StockVNQA_Lâm/train/data/{ticker}.csv")
        
        if len(df) < 60:  # Cần ít nhất 60 dòng dữ liệu
            return None
        
        # Kiểm tra dữ liệu đầu vào
        if 'open' not in df.columns:
            # Không có cột 'open', vẫn sử dụng features hiện tại
            features = ['close', 'volume', 'high', 'low']
        else:
            # Có cột 'open', thêm vào features
            features = ['close', 'open', 'volume', 'high', 'low']
        
        # --- PHẦN 1: TÍNH TOÁN CÁC FEATURES MỚI ---
        # Tạo bản sao để không làm ảnh hưởng đến dữ liệu gốc
        df_features = df.copy()
        
        # KIỂM TRA VÀ XỬ LÝ GIÁ TRỊ KHÔNG HỢP LỆ TRONG DỮ LIỆU GỐC
        # Thay thế các giá trị âm hoặc 0 trong volume bằng giá trị nhỏ
        if (df_features['volume'] <= 0).any():
            min_positive_volume = df_features.loc[df_features['volume'] > 0, 'volume'].min()
            df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
            
        # Đảm bảo giá không có giá trị âm
        for price_col in ['close', 'high', 'low']:
            if (df_features[price_col] <= 0).any():
                print(f"⚠️ {ticker} có giá {price_col} không hợp lệ (âm hoặc bằng 0)")
                return None
        
        # Tính toán các đặc trưng tương đối một cách an toàn
        df_features['daily_return'] = df_features['close'].pct_change().replace([np.inf, -np.inf], np.nan)
        df_features['high_low_range'] = ((df_features['high'] - df_features['low']) / df_features['close']).replace([np.inf, -np.inf], np.nan)
        
        # Nếu có cột open thì tính thêm đặc trưng close_to_open
        if 'open' in df.columns:
            df_features['close_to_open'] = ((df_features['close'] - df_features['open']) / df_features['open']).replace([np.inf, -np.inf], np.nan)
        
        # Tính toán các chỉ báo trung bình động
        df_features['ma5'] = df_features['close'].rolling(window=5).mean()
        df_features['ma20'] = df_features['close'].rolling(window=20).mean()
        
        # Xử lý trường hợp chia cho 0 hoặc NaN
        ma5 = df_features['ma5']
        ma20 = df_features['ma20']
        mask = (ma20 != 0) & ma20.notnull() & ma5.notnull()
        df_features['ma_ratio'] = np.nan
        df_features.loc[mask, 'ma_ratio'] = ma5.loc[mask] / ma20.loc[mask]
        
        # Tính toán độ biến động (volatility) an toàn
        df_features['volatility_5d'] = df_features['daily_return'].rolling(window=5).std().replace([np.inf, -np.inf], np.nan)
        
        # Phân tích khối lượng
        df_features['volume_ma5'] = df_features['volume'].rolling(window=5).mean()
        
        # Xử lý trường hợp chia cho 0 hoặc NaN
        vol = df_features['volume']
        vol_ma5 = df_features['volume_ma5']
        mask = (vol_ma5 != 0) & vol_ma5.notnull() & vol.notnull()
        df_features['volume_ratio'] = np.nan
        df_features.loc[mask, 'volume_ratio'] = vol.loc[mask] / vol_ma5.loc[mask]
        
        # Xử lý giá trị thiếu từ việc tính toán các chỉ báo - sửa warning
        df_features = df_features.ffill()  # Thay vì fillna(method='ffill')
        df_features = df_features.fillna(0)  # Điền 0 cho các giá trị NaN còn lại ở đầu
        
        # --- PHẦN 2: CHUẨN BỊ DỮ LIỆU ĐẦU VÀO ---
        # Danh sách các đặc trưng sẽ sử dụng, bao gồm cả đặc trưng gốc và đặc trưng mới
        extended_features = features.copy()  # Giữ các đặc trưng gốc
        
        # Thêm các đặc trưng mới đã tính toán
        additional_features = [
            'daily_return', 'high_low_range', 'ma_ratio', 
            'volatility_5d', 'volume_ratio'
        ]
        
        # Thêm close_to_open nếu có dữ liệu open
        if 'open' in df.columns:
            additional_features.append('close_to_open')
            
        # Kết hợp tất cả đặc trưng và kiểm tra giá trị không hợp lệ
        for feat in additional_features:
            if feat in df_features.columns:  # Chỉ thêm feature nếu tồn tại
                # Kiểm tra giá trị không hợp lệ
                if df_features[feat].isnull().any() or np.isinf(df_features[feat]).any():
                    print(f"⚠️ {ticker} có giá trị NaN hoặc Inf trong feature {feat}. Thay thế bằng 0.")
                    df_features[feat] = df_features[feat].replace([np.inf, -np.inf], 0).fillna(0)
                extended_features.append(feat)
        
        # KIỂM TRA LẦN CUỐI TRƯỚC KHI CHUẨN HÓA
        # Thay thế bất kỳ giá trị không hợp lệ còn lại
        df_check = df_features[extended_features]
        if df_check.isnull().any().any() or np.isinf(df_check).any().any():
            print(f"⚠️ {ticker} vẫn có giá trị không hợp lệ. Làm sạch dữ liệu...")
            df_check = df_check.replace([np.inf, -np.inf], 0)
            df_check = df_check.fillna(0)
            df_features[extended_features] = df_check
        
        # Chuẩn hóa tất cả đặc trưng
        scaler = MinMaxScaler()
        try:
            scaled_data = scaler.fit_transform(df_features[extended_features])
        except Exception as scale_error:
            print(f"❌ {ticker} - Lỗi khi chuẩn hóa dữ liệu: {str(scale_error)}")
            # Hiển thị thêm thông tin để debug
            print(f"Kiểm tra giá trị không hợp lệ: {df_features[extended_features].describe()}")
            return None

        # --- PHẦN 3: TẠO CHUỖI DỮ LIỆU CHO LSTM ---
        def create_dataset(data, window_size=30):
            X, y = [], []
            for i in range(len(data) - window_size):
                X.append(data[i:i+window_size])
                y.append(data[i+window_size, 0])  # Vẫn dự đoán close (cột đầu tiên)
            return np.array(X), np.array(y)

        window_size = 30
        X, y = create_dataset(scaled_data, window_size)

        # --- PHẦN 4: CHIA DỮ LIỆU TRAIN/TEST ---
        split = int(len(X) * 0.8)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # --- PHẦN 5: XÂY DỰNG VÀ HUẤN LUYỆN MÔ HÌNH ---
        model = Sequential()
        model.add(LSTM(64, input_shape=(window_size, X.shape[2])))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

        # --- PHẦN 6: DỰ ĐOÁN VÀ ĐÁNH GIÁ ---
        y_pred = model.predict(X_test)
        
        # Chuyển đổi giá trị về thang đo gốc
        # Tạo mảng với giá trị dự đoán ở vị trí đầu tiên và 0 ở các vị trí còn lại
        y_pred_full = np.zeros((len(y_pred), len(extended_features)))
        y_pred_full[:, 0] = y_pred.flatten()  # Đặt giá trị dự đoán vào cột đầu tiên (close)
        
        # Tương tự với giá trị thực tế
        y_test_full = np.zeros((len(y_test), len(extended_features)))
        y_test_full[:, 0] = y_test  # Đặt giá trị thực tế vào cột đầu tiên (close)
        
        # Inverse transform để lấy giá trị gốc
        y_pred_inv = scaler.inverse_transform(y_pred_full)[:, 0]
        y_test_inv = scaler.inverse_transform(y_test_full)[:, 0]

        # Tính RMSE
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
        
        # Tính R²
        r2 = r2_score(y_test_inv, y_pred_inv)
        
        # Tính Direction Accuracy
        start_idx = window_size + split
        actual_current_prices = df['close'].values[start_idx:start_idx+len(y_test_inv)-1]
        
        # Đảm bảo các mảng có cùng kích thước
        min_len = min(len(y_test_inv)-1, len(actual_current_prices))
        
        # Tính hướng giá thực tế và dự đoán, chỉ dùng phần chồng lấp
        actual_direction = (y_test_inv[1:min_len+1] > actual_current_prices[:min_len]).astype(int)
        predicted_direction = (y_pred_inv[1:min_len+1] > actual_current_prices[:min_len]).astype(int)
        
        # Tính direction accuracy
        direction_accuracy = np.mean(actual_direction == predicted_direction)
        
        # --- PHẦN 7: DỰ ĐOÁN GIÁ CHO NGÀY TIẾP THEO ---
        last_seq = scaled_data[-window_size:]
        last_seq = last_seq.reshape((1, window_size, len(extended_features)))
        next_day_scaled = model.predict(last_seq)
        
        # Tạo mảng đầy đủ để inverse transform
        next_day_full = np.zeros((1, len(extended_features)))
        next_day_full[0, 0] = next_day_scaled[0, 0]  # Đặt giá trị dự đoán vào cột đầu tiên
        
        next_day_price = scaler.inverse_transform(next_day_full)[0][0]
        
        # Tính % thay đổi
        current_price = df['close'].iloc[-1]
        change_pct = ((next_day_price - current_price) / current_price) * 100
        
        # Tùy chọn: Vẽ biểu đồ so sánh giá thực tế và dự đoán
        try:
            plt.figure(figsize=(10, 6))
            plt.plot(y_test_inv, label='Giá thực tế', marker='o', markersize=3, alpha=0.7)
            plt.plot(y_pred_inv, label='Giá dự đoán', marker='x', markersize=3)
            plt.title(f'Dự đoán giá {ticker} bằng LSTM (RMSE: {rmse:.4f}, R²: {r2:.4f})')
            plt.xlabel('Ngày kiểm tra')
            plt.ylabel('Giá đóng cửa')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            
            import os
            os.makedirs("E:/Project/StockVNQA_Lâm/train/charts", exist_ok=True)
            plt.savefig(f"E:/Project/StockVNQA_Lâm/train/charts/{ticker}_prediction.png")
            plt.show()
            plt.close()
        except Exception as chart_error:
            print(f"⚠️ Không thể vẽ biểu đồ cho {ticker}: {str(chart_error)}")
        
        return {
            'ticker': ticker,
            'total_days': len(df),
            'train_days': len(y_train),
            'test_days': len(y_test),
            'rmse': rmse,
            'r2': r2,
            'direction_accuracy': direction_accuracy,
            'current_price': current_price,
            'predicted_price': next_day_price,
            'change_pct': change_pct,
            'trend': 'UP' if change_pct > 0 else 'DOWN'
        }
        
    except Exception as e:
        print(f"❌ Lỗi xử lý {ticker} với LSTM: {str(e)}")
        import traceback
        traceback.print_exc()
        return None


import json
def load_tickers_from_file(filename, base_path="E:/Project/StockVNQA_Lâm/train/"):
    file_path = os.path.join(base_path, filename)
    with open(file_path, 'r') as f:
        return [line.strip() for line in f]
    






In [21]:
print(train("ACB"))

c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
{'ticker': 'ACB', 'total_days': 4605, 'train_days': 3660, 'test_days': 915, 'rmse': np.float64(0.31587244597363195), 'r2': 0.9916877851324002, 'direction_accuracy': np.float64(0.5054704595185996), 'current_price': np.float64(25.6), 'predicted_price': np.float64(25.34603397369385), 'change_pct': np.float64(-0.9920547902584065), 'trend': 'DOWN'}


R2 TICKERS THẤP NHẤT (THEO XGBOOST)

In [14]:
results = []

bottom_r2_tickers = load_tickers_from_file("bottom_r2_tickers.txt")
print(bottom_r2_tickers)
for ticker in bottom_r2_tickers:
    result = train(ticker)
    if result:
        results.append(result)


# Tạo DataFrame từ kết quả
result_df = pd.DataFrame(results)

# Hiển thị kết quả tổng hợp
print("\n=== KẾT QUẢ TỔNG HỢP ===")
print(f"Tổng số mã xử lý thành công: {len(result_df)}")
print(f"R² trung bình: {result_df['r2'].mean():.4f}")
print(f"RMSE trung bình: {result_df['rmse'].mean():.4f}")
print(f"Độ chính xác dự đoán hướng trung bình: {result_df['direction_accuracy'].mean():.4f}")
print(result_df[['ticker', 'rmse', 'r2', 'direction_accuracy', 'total_days']])
print(result_df["total_days"].sum())
print(result_df["train_days"].sum())
print(result_df["test_days"].sum())

['TMB', 'DDG', 'AAT', 'RCD', 'GH3', 'BSA', 'THM', 'HTR', 'FRT', 'SBG']


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
⚠️ HTR có giá close không hợp lệ (âm hoặc bằng 0)


c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

=== KẾT QUẢ TỔNG HỢP ===
Tổng số mã xử lý thành công: 8
R² trung bình: -2.5370
RMSE trung bình: 1.6311
Độ chính xác dự đoán hướng trung bình: 0.4565
  ticker      rmse         r2  direction_accuracy  total_days
0    TMB  2.119154   0.979336            0.482927        2081
1    DDG  0.203651   0.933120            0.565495        1599
2    AAT  0.092473   0.877254            0.402985        1036
3    GH3  0.035424 -24.029927            0.026490         790
4    BSA  0.550703   0.855705            0.708978        1650
5    THM  0.537638  -1.482679            0.400000         306
6    FRT  9.155113   0.859063            0.534682        1764
7    SBG  0.354666   0.712271            0.530303         361
9587
7474
1873


R2 TICKERS CAO NHẤT

In [16]:
results = []

top_r2_tickers = load_tickers_from_file("top_r2_tickers.txt")
print(top_r2_tickers)
for ticker in top_r2_tickers:
    result = train(ticker)
    if result:
        results.append(result)


# Tạo DataFrame từ kết quả
result_df = pd.DataFrame(results)

# Hiển thị kết quả tổng hợp
print("\n=== KẾT QUẢ TỔNG HỢP ===")
print(f"Tổng số mã xử lý thành công: {len(result_df)}")
print(f"R² trung bình: {result_df['r2'].mean():.4f}")
print(f"RMSE trung bình: {result_df['rmse'].mean():.4f}")
print(f"Độ chính xác dự đoán hướng trung bình: {result_df['direction_accuracy'].mean():.4f}")
print(result_df[['ticker', 'rmse', 'r2', 'direction_accuracy','total_days']])
print(result_df["total_days"].sum())
print(result_df["train_days"].sum())
print(result_df["test_days"].sum())

['DTL', 'TSC', 'TTF', 'VC2', 'S27', 'TBX', 'HTL', 'SRA', 'KMR', 'CVN']


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


...

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

=== KẾT QUẢ TỔNG HỢP ===
Tổng số mã xử lý thành công: 10
R² trung bình: 0.8835
RMSE trung bình: 0.3459
Độ chính xác dự đoán hướng trung bình: 0.5515
  ticker      rmse        r2  direction_accuracy  total_days
0    DTL  0.529572  0.996462            0.462483        3698
1    TSC  0.250044  0.996529            0.580275        4392
2    TTF  0.235822  0.995327            0.450820        4303
3    VC2  0.839480  0.996566            0.595838        4596
4    S27  0.198108  0.995330            0.407225        3076
5    TBX  0.247334 -0.116616            0.989704        3919
6    HTL  0.506987  0.994458            0.399154        3577
7    SRA  0.221267  0.994088            0.564593        4214
8    KMR  0.144852  0.991210            0.613636        4215
9    CVN  0.285215  0.991590            0.450867        3495
39485
31344
7841


RMSE TICKERS THAP NHAT

In [17]:
results = []

low_rmse_tickers = load_tickers_from_file("lowest_rmse_tickers.txt")
print(low_rmse_tickers)
for ticker in low_rmse_tickers:
    result = train(ticker)
    if result:
        results.append(result)


# Tạo DataFrame từ kết quả
result_df = pd.DataFrame(results)

# Hiển thị kết quả tổng hợp
print("\n=== KẾT QUẢ TỔNG HỢP ===")
print(f"Tổng số mã xử lý thành công: {len(result_df)}")
print(f"R² trung bình: {result_df['r2'].mean():.4f}")
print(f"RMSE trung bình: {result_df['rmse'].mean():.4f}")
print(f"Độ chính xác dự đoán hướng trung bình: {result_df['direction_accuracy'].mean():.4f}")
print(result_df[['ticker', 'rmse', 'r2', 'direction_accuracy', 'total_days']])
print(result_df["total_days"].sum())
print(result_df["train_days"].sum())
print(result_df["test_days"].sum())

['UMC', 'FCC', 'DPS', 'V11', 'ACM', 'NHP', 'HKB', 'CAD', 'G20', 'PVH']
⚠️ UMC có giá close không hợp lệ (âm hoặc bằng 0)
⚠️ FCC có giá close không hợp lệ (âm hoặc bằng 0)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
⚠️ V11 có giá close không hợp lệ (âm hoặc bằng 0)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.02' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer i

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


...

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in 

⚠️ CAD có giá close không hợp lệ (âm hoặc bằng 0)
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

=== KẾT QUẢ TỔNG HỢP ===
Tổng số mã xử lý thành công: 6
R² trung bình: 0.7859
RMSE trung bình: 0.0588
Độ chính xác dự đoán hướng trung bình: 0.3863
  ticker      rmse        r2  direction_accuracy  total_days
0    DPS  0.068551  0.809584            0.314978        2305
1    ACM  0.049990  0.766259            0.427938        2289
2    NHP  0.057216  0.900846            0.183983        2341
3    HKB  0.076576  0.456882            0.330472        2362
4    G20  0.050678  0.844228            0.295745        2381
5    PVH  0.049497  0.937466            0.764543        1839
13517
10667
2670


RMSE CAO NHAT

In [18]:
results = []

high_rmse_tickers = load_tickers_from_file("highest_rmse_tickers.txt")
print(high_rmse_tickers)
for ticker in high_rmse_tickers:
    result = train(ticker)
    if result:
        results.append(result)


# Tạo DataFrame từ kết quả
result_df = pd.DataFrame(results)

# Hiển thị kết quả tổng hợp
print("\n=== KẾT QUẢ TỔNG HỢP ===")
print(f"Tổng số mã xử lý thành công: {len(result_df)}")
print(f"R² trung bình: {result_df['r2'].mean():.4f}")
print(f"RMSE trung bình: {result_df['rmse'].mean():.4f}")
print(f"Độ chính xác dự đoán hướng trung bình: {result_df['direction_accuracy'].mean():.4f}")
print(result_df[['ticker', 'rmse', 'r2', 'direction_accuracy', 'total_days']])
print(result_df["total_days"].sum())
print(result_df["train_days"].sum())
print(result_df["test_days"].sum())

['HGM', 'KSV', 'VE4', 'CMF', 'FRT', 'WCS', 'MCH', 'PDN', 'TOS', 'AGX']


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


...

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
⚠️ AGX có giá close không hợp lệ (âm hoặc bằng 0)

=== KẾT QUẢ TỔNG HỢP ===
Tổng số mã xử lý thành công: 9
R² trung bình: 0.9637
RMSE trung bình: 9.0405
Độ chính xác dự đoán hướng trung bình: 0.5621
  ticker       rmse        r2  direction_accuracy  total_days
0    HGM  15.883857  0.966474            0.700787        3843
1    KSV  13.512932  0.959116            0.451389        2194
2    VE4   6.361411  0.994757            0.555938        2939
3    CMF  15.588533  0.935999            0.698795        2106
4    FRT   3.142394  0.983396            0.537572        1764
5    WCS   8.148068  0.984581            0.394958        3605
6    MCH   3.031451  0.994174            0.506083        2088
7    PDN   5.847358  0.936612            0.651719        3379
8    TOS   9.848429  0.918499            0.561798         921
22839
18052
4517


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01


Directional Acc Thap nhat

In [19]:
results = []

low_dir_tickers = load_tickers_from_file("lowest_dir_accuracy_tickers.txt")
print(low_dir_tickers)
for ticker in low_dir_tickers:
    result = train(ticker)
    if result:
        results.append(result)


# Tạo DataFrame từ kết quả
result_df = pd.DataFrame(results)

# Hiển thị kết quả tổng hợp
print("\n=== KẾT QUẢ TỔNG HỢP ===")
print(f"Tổng số mã xử lý thành công: {len(result_df)}")
print(f"R² trung bình: {result_df['r2'].mean():.4f}")
print(f"RMSE trung bình: {result_df['rmse'].mean():.4f}")
print(f"Độ chính xác dự đoán hướng trung bình: {result_df['direction_accuracy'].mean():.4f}")
print(result_df[['ticker', 'rmse', 'r2', 'direction_accuracy', 'total_days']])
print(result_df["total_days"].sum())
print(result_df["train_days"].sum())
print(result_df["test_days"].sum())

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.29' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.06' has dtype incompatible with int64, please explicitly c

['FCC', 'BHG', 'CDG', 'UMC', 'TQN', 'DKC', 'SDX', 'MPT', 'FGL', 'RCD']
⚠️ FCC có giá close không hợp lệ (âm hoặc bằng 0)
⚠️ BHG có giá close không hợp lệ (âm hoặc bằng 0)
⚠️ CDG có giá close không hợp lệ (âm hoặc bằng 0)
⚠️ UMC có giá close không hợp lệ (âm hoặc bằng 0)
⚠️ TQN có giá close không hợp lệ (âm hoặc bằng 0)
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
⚠️ SDX có giá close không hợp lệ (âm hoặc bằng 0)


c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
⚠️ FGL có giá close không hợp lệ (âm hoặc bằng 0)
⚠️ RCD có giá close không hợp lệ (âm hoặc bằng 0)

=== KẾT QUẢ TỔNG HỢP ===
Tổng số mã xử lý thành công: 2
R² trung bình: 0.4684
RMSE trung bình: 0.0619
Độ chính xác dự đoán hướng trung bình: 0.7667
  ticker      rmse        r2  direction_accuracy  total_days
0    DKC  0.042041  0.587706            0.964286        1011
1    MPT  0.081788  0.349110            0.569087        2166
3177
2492
625


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01


Directional Accuracy cao nhat

In [20]:
results = []

high_dir_tickers = load_tickers_from_file("highest_dir_accuracy_tickers.txt")
print(high_dir_tickers)
for ticker in high_dir_tickers:
    result = train(ticker)
    if result:
        results.append(result)


# Tạo DataFrame từ kết quả
result_df = pd.DataFrame(results)

# Hiển thị kết quả tổng hợp
print("\n=== KẾT QUẢ TỔNG HỢP ===")
print(f"Tổng số mã xử lý thành công: {len(result_df)}")
print(f"R² trung bình: {result_df['r2'].mean():.4f}")
print(f"RMSE trung bình: {result_df['rmse'].mean():.4f}")
print(f"Độ chính xác dự đoán hướng trung bình: {result_df['direction_accuracy'].mean():.4f}")
print(result_df[['ticker', 'rmse', 'r2', 'direction_accuracy', 'total_days']])
print(result_df["total_days"].sum())
print(result_df["train_days"].sum())
print(result_df["test_days"].sum())

['MEF', 'NQT', 'PTG', 'MTB', 'NAC', 'MND', 'HLT', 'QNU', 'VXP', 'GH3']
⚠️ MEF có giá close không hợp lệ (âm hoặc bằng 0)
⚠️ NQT có giá close không hợp lệ (âm hoặc bằng 0)
⚠️ PTG có giá close không hợp lệ (âm hoặc bằng 0)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.02' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1' has dtype incompatible with int64, please explicitly ca

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9204\788956016.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.08' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_features.loc[df_features['volume'] <= 0, 'volume'] = min_positive_volume * 0.01
c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
⚠️ MND có giá close không hợp lệ (âm hoặc bằng 0)


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
⚠️ QNU có giá close không hợp lệ (âm hoặc bằng 0)


c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


c:\Users\ADMIN\miniconda3\envs\stockqa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

=== KẾT QUẢ TỔNG HỢP ===
Tổng số mã xử lý thành công: 5
R² trung bình: -157268130452735406920171520.0000
RMSE trung bình: 0.3148
Độ chính xác dự đoán hướng trung bình: 0.5856
  ticker      rmse            r2  direction_accuracy  total_days
0    MTB  0.076273 -4.609199e+26            1.000000         903
1    NAC  0.023485  9.644980e-01            0.839779        1841
2    HLT  1.299436  6.995321e-01            0.081081        1143
3    VXP  0.128178 -3.254207e+26            0.987500        1235
4    GH3  0.046650 -4.240802e+01            0.019868         790
5912
4608
1154
